# This notebook uses the injury data scraped from Pro Sports Transactions, and both pulls game logs for all players in seasons where they were injured, and associates each injury with the game where the player was injured

In [1]:
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup, Comment
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import datetime
import unidecode
from googlesearch import search

mpl.rcParams['font.family'] = "serif"
mpl.rcParams['text.usetex'] = True
mpl.rcParams['ytick.right'] = True
mpl.rcParams['ytick.direction'] = "in"
mpl.rcParams['xtick.top'] = True
mpl.rcParams['xtick.direction'] = "in"
mpl.rcParams['axes.titlesize'] = 20
mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['legend.fontsize'] = 20
mpl.rcParams['figure.dpi'] = 100
mpl.rcParams['savefig.dpi'] = 100
mpl.rcParams['figure.figsize'] = (6,4)
plt.rcParams['axes.labelweight'] = 'bold'
mpl.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]    

In [2]:
class ScrapingError(Exception):
    pass

class player:
    
    def __init__(self,name=""):
        self.name = name
        self.stat_dict = {}
        self.play_str = ""
        self.initial = ""
        self.height = None
        self.weight = None
        self.pos = None
        self.pos3 = None
        self.shoots = None
        
    def get_play_str(self,verbose=False):
        
        """Get the 'player string', associated with a player used by basketball reference in the url for all of their
        info. Generally it's the first 5 letters of the players last name and the second two of their first, plus a
        numerical counter which counts up when players have matching strings. Aliases can causes issues with this,
        e.g. Metta World Peace/Ron Artest causes an issue with matching the strings"""
        
        name = self.name
            
        name_proc = name.replace(".","")
        name_proc = name_proc.replace("'","")
        
        name_arr= name_proc.split()
        fname = name_arr[0]
        lname = ''.join(name_arr[1:])
        
        fname = fname.lower()
        lname = lname.lower()

        initial = lname[0]
        
        self.initial = initial
        
        play_num = 1
        play_count = '0' + str(play_num)

        player_found = False
        
        try:
        
            while not(player_found):
        
                url = "https://www.basketball-reference.com/players/{}/{}{}{}.html".\
                    format(initial,lname[:5],fname[:2],play_count)

                if verbose:
                    print(url)
            
                html = urlopen(url)
                soup = BeautifulSoup(html)

                #See if the player string is correct by matching the listed name to the inputted name
                name_str = unidecode.unidecode(soup.find_all('strong')[0].contents[0]) #Use unidecode to get rid of accents
                try:
                    birth_year = int(soup.find_all(id='necro-birth')[0].getText().split('\n')[-2][-4:]) #Pull player's birth year
                except IndexError:
                    birth_year = 2000
                #Also check if player is born after 1965 to avoid confusion with older players who have same name
                #Still won't help if two players have the same name in the current dataset
                if (name_str == name and birth_year > 1965):
                    if verbose:
                        print("Correct ID tag for {}".format(name))
                    player_found = True
                    self.play_str = lname[:5] + fname[:2] + play_count
                else:
                    #If this doesn't work, increment the counter at the end by 1
                    play_num += 1
                    play_count = '0' + str(play_num)
                    print("Incorrect ID tag for {}, trying counter {}".format(name,play_count))
                
                            
        except HTTPError:
            #Once we've failed at doing that, resort to just googling the player's name and basketball reference
            if verbose:
                print("Page not found, trying google")
            query = name + " basketball reference"
            strs = search(query, tld="com", num=1, stop=1)
            results = [s for s in strs]
            url = results[0]
            self.play_str = url.split('/')[-1][:-5] #Cut up url by / and pull all but last 5 chars of text after last /

            html = urlopen(url)
            soup = BeautifulSoup(html)
                
        for s in soup.find_all('p'): #Get player height and weight
            hw_stats = s.getText().find("cm")
            if hw_stats != -1:
                hw_str = s.getText()
                break
                
        for s in soup.find_all('p'):
            pos = s.getText().find("Position")
            if pos != -1:
                pos_shoots = s.getText()
                break
                    
        try:
            hw_str = hw_str.split('(')[-1] #Pull everything after the open parenses
            hw_arr = ''.join(filter(str.isdigit, hw_str)) #Pull out just the numbers

            self.height = int(hw_arr[:3]) #Assumes everyone is greater than 99 cm, so first 3 characters are height
            self.weight = int(hw_arr[3:])
        except NameError:
            self.height = 0
            self.weight = 0
            
        try:
            
            nl_split = pos_shoots.split('\n')
            pos_str = nl_split[4].strip()
            self.pos = pos_str
           
            #Quick and dirty alogrithm to classsify players as either 'Big', 'Guard', or 'Wing' depending on their
            #Listed position on Bball reference
            if pos_str.find('Center') != -1:
                pos3 = 'Big'
            elif pos_str.find('Small') != -1:
                pos3 = 'Wing'
            elif pos_str.find('Point') != -1:
                pos3 = 'Guard'
            elif pos_str.find('Shooting') != -1:
                pos3 = 'Wing'
            elif pos_str.find('Power') != -1:
                pos3 = 'Big'
            else:
                pos3 = 'None'
#             print("{} is catergorized as a {}\n".format(name,pos3))
            
            self.pos3 = pos3
            self.shoots = nl_split[-2].strip()
            
        except NameError:
            self.pos = 0
            self.pos3 = 0
            self.shoots = 0
            

        
    def get_gamelog(self,year,verbose=False):
        
        """Get log of all games played from a given year from basketball reference"""
        
        if year in self.stat_dict:
#             if verbose:
#                 print("\nStats already loaded in")
            return
        
        year = str(year)
        
        if self.play_str == "":
            self.get_play_str(verbose=verbose)     
        
        url_log = "https://www.basketball-reference.com/players/{}/{}/gamelog/{}"\
        .format(self.initial,self.play_str,year)
        
        if verbose:
            print("\nTrying to pull stats for {} from {}-{} season".format(self.name,int(year)-1,year))

        html = urlopen(url_log)
        soup = BeautifulSoup(html)

        headers = [th.getText() for th in soup.findAll('th',scope='col')]

        reg_s_body = soup.findAll('tbody')
        
        
        if reg_s_body == []:
            raise ScrapingError("No rows found")

        rows = reg_s_body[0].findAll('tr')
        
        # Playoff rows are going to be commented out in soup, so to get playoff rows we find commented out rows
        comment = soup.findAll(text=lambda text:isinstance(text, Comment))

        real = []
        for c in comment:
            if len(c)>500: #500 is currently a magic number as there are some shorter entries that are commented out
                real.append(c)
        try:
            commentSoup = BeautifulSoup(real[0])

            playoff_body = commentSoup.findAll('tbody')
            playoff_rows = playoff_body[0].findAll('tr')

            for i in range(len(playoff_rows)):
                rows.append(playoff_rows[i])

        except IndexError:
            if verbose:
                print("No playoff games for {}-{} season".format(int(year)-1,year))
                

        vals = [[td.getText() for td in rows[i].findAll('td')]
                for i in range(len(rows))]
        
        try:
            stats = pd.DataFrame(vals, columns = headers[1:])
            stats = stats.dropna(how='all').reset_index()
            stats['Age'] = stats['Age'].apply(age_to_num) #Turn age to numeric values
            stats['MP'] = stats['MP'].apply(MP_to_num) # Turn MP to numeric values
             
            self.stat_dict[year] = stats           

        except AssertionError:
            if vals[0][-1] in {'Inactive','Did Not Play','Not With Team','Did Not Dress'}:
                print("{} was inactive for the entire {}-{} season\n".format(self.name,int(year)-1,year))
                self.stat_dict[year] = pd.DataFrame(columns = headers[1:])
            else:
                raise Exception("An unknown error occurred")       
        
def gamelog_from_injury(player,entry,verbose=False):
    """Find the gamelog for a given year when the players was injured"""

    date = (entry['Date'].split('-'))

    y = int(date[0])
    m = int(date[1])
    d = int(date[2])

    inj_date = datetime.datetime(y,m,d)
    if inj_date < datetime.datetime(y,7,1):
        year = str(y)
    else:
        year = str(y+1)

    try: 
        player.get_gamelog(year,verbose=verbose)
        return player, inj_date, year
    except ScrapingError as e:
        raise e
    
def str2datetime(date):
    
    """Return a datetime object from a string given as y-m-d"""
    
    date_tmp = date.split('-')
    
    y = int(date_tmp[0])
    m = int(date_tmp[1])
    d = int(date_tmp[2])
    
    return datetime.datetime(y,m,d)

def find_inj_game(player,entry,verbose=False):
    
    try:
        player,inj_date,year = gamelog_from_injury(player,entry,verbose=verbose)
    except ScrapingError as e:
        raise e
    
    player,inj_date,year = gamelog_from_injury(player,entry,verbose=verbose)
    
    season_stats =  player.stat_dict[year]
    season_stats =  season_stats.loc[season_stats['MP'] != 0].reset_index()
    
    deltas = []

    for i,r in season_stats.iterrows():
    
        date = str2datetime(r['Date'])       

        diff = inj_date - date      
    
        if diff > datetime.timedelta(0):
            deltas.append([i,diff])
            
    if deltas == []:
        raise ValueError('Pre_season injury')
    else:
        i = deltas[-1][0]


    if i == season_stats.shape[0] - 1:
        return_date = datetime.datetime(int(year),8,1)
    else:
        return_date = str2datetime(season_stats.iloc[i+1]['Date'])
    return season_stats.iloc[i],date,return_date,year
        
#Simple functions to make decimal values from Bball reference table values
def age_to_num(age):
    yr = int(age[:2])
    frac = int(age[3:])/365.25    
    return yr+frac

def MP_to_num(MP):  
    if MP == None:
        return 0
    else:
        MP_arr = MP.split(':')
        mnts = int(MP_arr[0])
        secs = int(MP_arr[1])/60.
    return mnts+secs

In [3]:
df = pd.read_csv('nba_injs_2010-2020.csv')

injs = df.loc[df['Acquired'].isnull()].reset_index(drop=True)

#Nene's mononym is going cause a problem, so add his last name in
injs['Relinquished'].replace(to_replace='Nene',value='Nene Hilario',inplace=True)
player_names = injs.Relinquished.unique()

print("Found {} injuries listed for {} players".format(injs.shape[0],len(player_names)))

Found 13700 injuries listed for 1118 players


In [6]:
injs_df_arr = []
noinjs_df_arr = np.zeros((1,32))
players_failed = []

for name in player_names:
    
    player_injs = injs.loc[injs['Relinquished'] == name].reset_index()
    p_object = player(name)
   
    ret_game = datetime.datetime(1800,1,1)

    for i,r in player_injs.iterrows():
        if str2datetime(r['Date']) < ret_game:
            continue        
        try:             
            ret = find_inj_game(p_object,r,verbose=True)
        except HTTPError:
            print("\nCouldn't find page for player {}\n".format(name))
            continue
        except ValueError as e:
#             print("Injury number {} was a {}".format(i,e))
            continue
        except ScrapingError as e:
            print(e)
            continue           
        if ret == None:
            print("Query returned nothing")
        else:
            #Add injury games to array
            inj_game,date,ret_game,year = ret
            inj_arr = [p_object.name,p_object.height,p_object.weight,p_object.pos,p_object.pos3,p_object.shoots,inj_game['Date'],inj_game['Age'],inj_game['Opp'],
                       inj_game['MP'],inj_game['FG'],inj_game['FGA'],inj_game['FG%'],inj_game['3P'],inj_game['3PA']\
                       ,inj_game['3P%'],inj_game['FT'],inj_game['FTA'],inj_game['FT%'],inj_game['ORB'],inj_game['DRB']\
                       ,inj_game['TRB'],inj_game['AST'],inj_game['STL'],inj_game['BLK'],inj_game['TOV'],inj_game['PF'],\
                       inj_game['PTS'],inj_game['GmSc'],inj_game['+/-'],1,r['Notes']]
            injs_df_arr.append(inj_arr)
            p_object.stat_dict[year] = p_object.stat_dict[year].loc[p_object.stat_dict[year]['Date'] != inj_game['Date']]
            
    #Add non-injury games to array 
    
    years = [int(key) for key in p_object.stat_dict.keys()]
    
    if len(years)==0:
        players_failed.append(name)
        continue
    y_down = min(years)
    y_up = min(years)

    if y_down != 2011:
        while(y_down != 2011):
            y_down -= 1
            try:
                p_object.get_gamelog(str(y_down),verbose=True)
            except (ScrapingError,HTTPError):
                continue

    if y_up != 2020:
        while(y_up != 2020):
            y_up += 1
            try:
                p_object.get_gamelog(str(y_up),verbose=True)
            except (ScrapingError,HTTPError):
                continue
            
    p_object.stat_dict = {key: val for key, val in p_object.stat_dict.items() if len(val) > 0}
    if p_object.stat_dict == {}:
        players_failed.append(name)
        continue
                    
    games_played = [p_object.stat_dict[x].loc[p_object.stat_dict[x]['MP'] != 0] for x in p_object.stat_dict]
    all_games_df = pd.concat(games_played)
    N_games = all_games_df.shape[0]
    
    if (N_games == 0 or N_games == 1):
        players_failed.append(name)
        continue

    name_arr = [p_object.name for x in range(N_games)]
    height_arr = [p_object.height for x in range(N_games)]
    weight_arr = [p_object.weight for x in range(N_games)]
    pos_arr = [p_object.pos for x in range(N_games)]
    pos3_arr = [p_object.pos3 for x in range(N_games)]
    shoots_arr = [p_object.shoots for x in range(N_games)]
    
    was_inj_arr = np.zeros(N_games)
    note_arr = ["No injury" for x in range(N_games)]

    noinj_arr_tmp = np.transpose([name_arr,height_arr,weight_arr,pos_arr,pos3_arr,shoots_arr,all_games_df['Date'],all_games_df['Age'],all_games_df['Opp'],
                       all_games_df['MP'],all_games_df['FG'],all_games_df['FGA'],all_games_df['FG%'],all_games_df['3P'],all_games_df['3PA']\
                       ,all_games_df['3P%'],all_games_df['FT'],all_games_df['FTA'],all_games_df['FT%'],all_games_df['ORB'],all_games_df['DRB']\
                       ,all_games_df['TRB'],all_games_df['AST'],all_games_df['STL'],all_games_df['BLK'],all_games_df['TOV'],all_games_df['PF'],\
                      all_games_df['PTS'],all_games_df['GmSc'],all_games_df['+/-'],\
               was_inj_arr,note_arr])
    
    noinjs_df_arr = np.concatenate((noinjs_df_arr,noinj_arr_tmp))

https://www.basketball-reference.com/players/b/boozeca01.html
Correct ID tag for Carlos Boozer

Trying to pull stats for Carlos Boozer from 2010-2011 season

Trying to pull stats for Carlos Boozer from 2012-2013 season

Trying to pull stats for Carlos Boozer from 2013-2014 season

Trying to pull stats for Carlos Boozer from 2014-2015 season
No playoff games for 2014-2015 season

Trying to pull stats for Carlos Boozer from 2011-2012 season

Trying to pull stats for Carlos Boozer from 2015-2016 season

Trying to pull stats for Carlos Boozer from 2016-2017 season

Trying to pull stats for Carlos Boozer from 2017-2018 season

Trying to pull stats for Carlos Boozer from 2018-2019 season

Trying to pull stats for Carlos Boozer from 2019-2020 season
https://www.basketball-reference.com/players/j/jerebjo01.html
Correct ID tag for Jonas Jerebko

Trying to pull stats for Jonas Jerebko from 2010-2011 season
No playoff games for 2010-2011 season
Jonas Jerebko was inactive for the entire 2010-2011 


Trying to pull stats for Nick Collison from 2017-2018 season

Trying to pull stats for Nick Collison from 2018-2019 season

Trying to pull stats for Nick Collison from 2019-2020 season
https://www.basketball-reference.com/players/w/williel01.html
Correct ID tag for Elliot Williams

Trying to pull stats for Elliot Williams from 2010-2011 season
Elliot Williams was inactive for the entire 2010-2011 season


Trying to pull stats for Elliot Williams from 2011-2012 season
No playoff games for 2011-2012 season

Trying to pull stats for Elliot Williams from 2012-2013 season
No playoff games for 2012-2013 season
Elliot Williams was inactive for the entire 2012-2013 season


Trying to pull stats for Elliot Williams from 2013-2014 season
No playoff games for 2013-2014 season

Trying to pull stats for Elliot Williams from 2014-2015 season
No playoff games for 2014-2015 season

Trying to pull stats for Elliot Williams from 2015-2016 season
No playoff games for 2015-2016 season

Trying to pull sta


Trying to pull stats for Gani Lawal from 2012-2013 season

Trying to pull stats for Gani Lawal from 2013-2014 season

Trying to pull stats for Gani Lawal from 2014-2015 season

Trying to pull stats for Gani Lawal from 2015-2016 season

Trying to pull stats for Gani Lawal from 2016-2017 season

Trying to pull stats for Gani Lawal from 2017-2018 season

Trying to pull stats for Gani Lawal from 2018-2019 season

Trying to pull stats for Gani Lawal from 2019-2020 season
https://www.basketball-reference.com/players/j/jannima01.html
Correct ID tag for Matt Janning

Trying to pull stats for Matt Janning from 2010-2011 season
No playoff games for 2010-2011 season
Matt Janning was inactive for the entire 2010-2011 season


Trying to pull stats for Matt Janning from 2011-2012 season

Trying to pull stats for Matt Janning from 2012-2013 season

Trying to pull stats for Matt Janning from 2013-2014 season

Trying to pull stats for Matt Janning from 2014-2015 season

Trying to pull stats for Matt J

Incorrect ID tag for Maurice Williams, trying counter 04
https://www.basketball-reference.com/players/w/willima04.html
Incorrect ID tag for Maurice Williams, trying counter 05
https://www.basketball-reference.com/players/w/willima05.html
Incorrect ID tag for Maurice Williams, trying counter 06
https://www.basketball-reference.com/players/w/willima06.html
Page not found, trying google

Trying to pull stats for Maurice Williams from 2010-2011 season
No playoff games for 2010-2011 season

Trying to pull stats for Maurice Williams from 2011-2012 season

Trying to pull stats for Maurice Williams from 2012-2013 season
No playoff games for 2012-2013 season

Trying to pull stats for Maurice Williams from 2013-2014 season

Trying to pull stats for Maurice Williams from 2014-2015 season
No playoff games for 2014-2015 season

Trying to pull stats for Maurice Williams from 2015-2016 season

Trying to pull stats for Maurice Williams from 2016-2017 season
No playoff games for 2016-2017 season
Mauric


Trying to pull stats for Quincy Pondexter from 2019-2020 season
https://www.basketball-reference.com/players/w/wrighan01.html
Correct ID tag for Antoine Wright

Trying to pull stats for Antoine Wright from 2010-2011 season
No playoff games for 2010-2011 season

Trying to pull stats for Antoine Wright from 2011-2012 season

Trying to pull stats for Antoine Wright from 2012-2013 season

Trying to pull stats for Antoine Wright from 2013-2014 season

Trying to pull stats for Antoine Wright from 2014-2015 season

Trying to pull stats for Antoine Wright from 2015-2016 season

Trying to pull stats for Antoine Wright from 2016-2017 season

Trying to pull stats for Antoine Wright from 2017-2018 season

Trying to pull stats for Antoine Wright from 2018-2019 season

Trying to pull stats for Antoine Wright from 2019-2020 season
https://www.basketball-reference.com/players/r/randoan01.html
Correct ID tag for Anthony Randolph

Trying to pull stats for Anthony Randolph from 2010-2011 season
No playo

...


Trying to pull stats for Erick Dampier from 2015-2016 season

Trying to pull stats for Erick Dampier from 2016-2017 season

Trying to pull stats for Erick Dampier from 2017-2018 season

Trying to pull stats for Erick Dampier from 2018-2019 season

Trying to pull stats for Erick Dampier from 2019-2020 season
https://www.basketball-reference.com/players/p/pargoje01.html
Correct ID tag for Jeremy Pargo

Trying to pull stats for Jeremy Pargo from 2011-2012 season

Trying to pull stats for Jeremy Pargo from 2012-2013 season
No playoff games for 2012-2013 season

Trying to pull stats for Jeremy Pargo from 2010-2011 season

Trying to pull stats for Jeremy Pargo from 2013-2014 season

Trying to pull stats for Jeremy Pargo from 2014-2015 season

Trying to pull stats for Jeremy Pargo from 2015-2016 season

Trying to pull stats for Jeremy Pargo from 2016-2017 season

Trying to pull stats for Jeremy Pargo from 2017-2018 season

Trying to pull stats for Jeremy Pargo from 2018-2019 season

Trying t


Trying to pull stats for James Jones (b) from 2019-2020 season
https://www.basketball-reference.com/players/g/griffbl01.html
Correct ID tag for Blake Griffin

Trying to pull stats for Blake Griffin from 2012-2013 season

Trying to pull stats for Blake Griffin from 2013-2014 season

Trying to pull stats for Blake Griffin from 2014-2015 season

Trying to pull stats for Blake Griffin from 2015-2016 season

Trying to pull stats for Blake Griffin from 2016-2017 season

Trying to pull stats for Blake Griffin from 2017-2018 season
No playoff games for 2017-2018 season

Trying to pull stats for Blake Griffin from 2018-2019 season

Trying to pull stats for Blake Griffin from 2019-2020 season
No playoff games for 2019-2020 season

Trying to pull stats for Blake Griffin from 2011-2012 season

Trying to pull stats for Blake Griffin from 2010-2011 season
No playoff games for 2010-2011 season
https://www.basketball-reference.com/players/m/melofa01.html
Correct ID tag for Fab Melo

Trying to pull st

No playoff games for 2012-2013 season

Trying to pull stats for Diante Garrett from 2011-2012 season

Trying to pull stats for Diante Garrett from 2010-2011 season

Trying to pull stats for Diante Garrett from 2013-2014 season
No playoff games for 2013-2014 season

Trying to pull stats for Diante Garrett from 2014-2015 season

Trying to pull stats for Diante Garrett from 2015-2016 season

Trying to pull stats for Diante Garrett from 2016-2017 season

Trying to pull stats for Diante Garrett from 2017-2018 season

Trying to pull stats for Diante Garrett from 2018-2019 season

Trying to pull stats for Diante Garrett from 2019-2020 season
https://www.basketball-reference.com/players/b/bazemke01.html
Correct ID tag for Kent Bazemore

Trying to pull stats for Kent Bazemore from 2012-2013 season

Trying to pull stats for Kent Bazemore from 2013-2014 season
No playoff games for 2013-2014 season

Trying to pull stats for Kent Bazemore from 2015-2016 season

Trying to pull stats for Kent Bazemor


Trying to pull stats for Kendall Marshall from 2018-2019 season

Trying to pull stats for Kendall Marshall from 2019-2020 season
https://www.basketball-reference.com/players/j/jonesde01.html
Correct ID tag for DeQuan Jones

Trying to pull stats for DeQuan Jones from 2012-2013 season
No playoff games for 2012-2013 season

Trying to pull stats for DeQuan Jones from 2011-2012 season

Trying to pull stats for DeQuan Jones from 2010-2011 season

Trying to pull stats for DeQuan Jones from 2013-2014 season

Trying to pull stats for DeQuan Jones from 2014-2015 season

Trying to pull stats for DeQuan Jones from 2015-2016 season

Trying to pull stats for DeQuan Jones from 2016-2017 season

Trying to pull stats for DeQuan Jones from 2017-2018 season

Trying to pull stats for DeQuan Jones from 2018-2019 season

Trying to pull stats for DeQuan Jones from 2019-2020 season
https://www.basketball-reference.com/players/f/fieldla01.html
Correct ID tag for Landry Fields

Trying to pull stats for Landry 

KeyboardInterrupt: 

In [7]:
injs_df = pd.DataFrame(injs_df_arr,columns=["Name","Height [cm]","Weight [kg]","Position","Pos3","Shoots","Date","Age","Opp","MP","FG","FGA","FG%","3P","3PA",\
                                            "3P%","FT","FTA","FT%","ORB","DRB","TRB","AST","STL","BLK","TOV","PF","PTS","GmSc","+/-","Was_Injured?","Notes"])
no_injs_df = pd.DataFrame(noinjs_df_arr,columns=["Name","Height [cm]","Weight [kg]","Position","Pos3","Shoots","Date","Age","Opp","MP","FG","FGA","FG%","3P","3PA",\
                                            "3P%","FT","FTA","FT%","ORB","DRB","TRB","AST","STL","BLK","TOV","PF","PTS","GmSc","+/-","Was_Injured?","Notes"])

In [8]:
print(injs_df.shape,no_injs_df.shape)

(4674, 32) (141073, 32)


In [20]:
np.where(player_names=='Isaac Bonga')

(array([1117]),)

## Continue scrape if an error occurs

In [19]:
for name in player_names[966:]:
    
    player_injs = injs.loc[injs['Relinquished'] == name].reset_index()
    p_object = player(name)
   
    ret_game = datetime.datetime(1800,1,1)

    for i,r in player_injs.iterrows():
        if str2datetime(r['Date']) < ret_game:
            continue        
        try:             
            ret = find_inj_game(p_object,r,verbose=True)
        except HTTPError:
            print("\nCouldn't find page for player {}\n".format(name))
            continue
        except ValueError as e:
#             print("Injury number {} was a {}".format(i,e))
            continue
        except ScrapingError as e:
            print(e)
            continue           
        if ret == None:
            print("Query returned nothing")
        else:
            #Add injury games to array
            inj_game,date,ret_game,year = ret
            inj_arr = [p_object.name,p_object.height,p_object.weight,p_object.pos,p_object.pos3,p_object.shoots,inj_game['Date'],inj_game['Age'],inj_game['Opp'],
                       inj_game['MP'],inj_game['FG'],inj_game['FGA'],inj_game['FG%'],inj_game['3P'],inj_game['3PA']\
                       ,inj_game['3P%'],inj_game['FT'],inj_game['FTA'],inj_game['FT%'],inj_game['ORB'],inj_game['DRB']\
                       ,inj_game['TRB'],inj_game['AST'],inj_game['STL'],inj_game['BLK'],inj_game['TOV'],inj_game['PF'],\
                       inj_game['PTS'],inj_game['GmSc'],inj_game['+/-'],1,r['Notes']]
            injs_df_arr.append(inj_arr)
            p_object.stat_dict[year] = p_object.stat_dict[year].loc[p_object.stat_dict[year]['Date'] != inj_game['Date']]
            
    #Add non-injury games to array 
    
    years = [int(key) for key in p_object.stat_dict.keys()]
    
    if len(years)==0:
        players_failed.append(name)
        continue
    y_down = min(years)
    y_up = min(years)

    if y_down != 2011:
        while(y_down != 2011):
            y_down -= 1
            try:
                p_object.get_gamelog(str(y_down),verbose=True)
            except (ScrapingError,HTTPError):
                continue

    if y_up != 2020:
        while(y_up != 2020):
            y_up += 1
            try:
                p_object.get_gamelog(str(y_up),verbose=True)
            except (ScrapingError,HTTPError):
                continue
            
    p_object.stat_dict = {key: val for key, val in p_object.stat_dict.items() if len(val) > 0}
    if p_object.stat_dict == {}:
        players_failed.append(name)
        continue
                    
    games_played = [p_object.stat_dict[x].loc[p_object.stat_dict[x]['MP'] != 0] for x in p_object.stat_dict]
    all_games_df = pd.concat(games_played)
    N_games = all_games_df.shape[0]
    
    if (N_games == 0 or N_games == 1):
        players_failed.append(name)
        continue

    name_arr = [p_object.name for x in range(N_games)]
    height_arr = [p_object.height for x in range(N_games)]
    weight_arr = [p_object.weight for x in range(N_games)]
    pos_arr = [p_object.pos for x in range(N_games)]
    pos3_arr = [p_object.pos3 for x in range(N_games)]
    shoots_arr = [p_object.shoots for x in range(N_games)]
    
    was_inj_arr = np.zeros(N_games)
    note_arr = ["No injury" for x in range(N_games)]

    noinj_arr_tmp = np.transpose([name_arr,height_arr,weight_arr,pos_arr,pos3_arr,shoots_arr,all_games_df['Date'],all_games_df['Age'],all_games_df['Opp'],
                       all_games_df['MP'],all_games_df['FG'],all_games_df['FGA'],all_games_df['FG%'],all_games_df['3P'],all_games_df['3PA']\
                       ,all_games_df['3P%'],all_games_df['FT'],all_games_df['FTA'],all_games_df['FT%'],all_games_df['ORB'],all_games_df['DRB']\
                       ,all_games_df['TRB'],all_games_df['AST'],all_games_df['STL'],all_games_df['BLK'],all_games_df['TOV'],all_games_df['PF'],\
                      all_games_df['PTS'],all_games_df['GmSc'],all_games_df['+/-'],\
               was_inj_arr,note_arr])
    
    noinjs_df_arr = np.concatenate((noinjs_df_arr,noinj_arr_tmp))

https://www.basketball-reference.com/players/m/masonfr01.html
Correct ID tag for Frank Mason

Trying to pull stats for Frank Mason from 2017-2018 season
No playoff games for 2017-2018 season

Trying to pull stats for Frank Mason from 2019-2020 season
No playoff games for 2019-2020 season

Trying to pull stats for Frank Mason from 2016-2017 season

Trying to pull stats for Frank Mason from 2015-2016 season

Trying to pull stats for Frank Mason from 2014-2015 season

Trying to pull stats for Frank Mason from 2013-2014 season

Trying to pull stats for Frank Mason from 2012-2013 season

Trying to pull stats for Frank Mason from 2011-2012 season

Trying to pull stats for Frank Mason from 2010-2011 season

Trying to pull stats for Frank Mason from 2018-2019 season
No playoff games for 2018-2019 season
https://www.basketball-reference.com/players/c/chrisma01.html
Correct ID tag for Marquese Chriss

Trying to pull stats for Marquese Chriss from 2017-2018 season
No playoff games for 2017-2018 s

Page not found, trying google

Trying to pull stats for Xavier Munford from 2017-2018 season
No playoff games for 2017-2018 season

Trying to pull stats for Xavier Munford from 2016-2017 season

Trying to pull stats for Xavier Munford from 2015-2016 season

Trying to pull stats for Xavier Munford from 2014-2015 season

Trying to pull stats for Xavier Munford from 2013-2014 season

Trying to pull stats for Xavier Munford from 2012-2013 season

Trying to pull stats for Xavier Munford from 2011-2012 season

Trying to pull stats for Xavier Munford from 2010-2011 season

Trying to pull stats for Xavier Munford from 2018-2019 season

Trying to pull stats for Xavier Munford from 2019-2020 season
https://www.basketball-reference.com/players/y/yabusgu01.html
Correct ID tag for Guerschon Yabusele

Trying to pull stats for Guerschon Yabusele from 2017-2018 season

Trying to pull stats for Guerschon Yabusele from 2018-2019 season

Trying to pull stats for Guerschon Yabusele from 2016-2017 season




Trying to pull stats for Tyrone Wallace from 2014-2015 season

Trying to pull stats for Tyrone Wallace from 2013-2014 season

Trying to pull stats for Tyrone Wallace from 2012-2013 season

Trying to pull stats for Tyrone Wallace from 2011-2012 season

Trying to pull stats for Tyrone Wallace from 2010-2011 season

Trying to pull stats for Tyrone Wallace from 2019-2020 season
No playoff games for 2019-2020 season
https://www.basketball-reference.com/players/o/oubreke01.html
Incorrect ID tag for Kelly Oubre Jr., trying counter 02
https://www.basketball-reference.com/players/o/oubreke02.html
Page not found, trying google

Trying to pull stats for Kelly Oubre Jr. from 2017-2018 season

Trying to pull stats for Kelly Oubre Jr. from 2018-2019 season
No playoff games for 2018-2019 season

Trying to pull stats for Kelly Oubre Jr. from 2019-2020 season
No playoff games for 2019-2020 season

Trying to pull stats for Kelly Oubre Jr. from 2016-2017 season

Trying to pull stats for Kelly Oubre Jr. 


Trying to pull stats for Robert Williams III from 2014-2015 season

Trying to pull stats for Robert Williams III from 2013-2014 season

Trying to pull stats for Robert Williams III from 2012-2013 season

Trying to pull stats for Robert Williams III from 2011-2012 season

Trying to pull stats for Robert Williams III from 2010-2011 season
https://www.basketball-reference.com/players/v/vandeja01.html
Correct ID tag for Jarred Vanderbilt

Trying to pull stats for Jarred Vanderbilt from 2018-2019 season

Trying to pull stats for Jarred Vanderbilt from 2017-2018 season

Trying to pull stats for Jarred Vanderbilt from 2016-2017 season

Trying to pull stats for Jarred Vanderbilt from 2015-2016 season

Trying to pull stats for Jarred Vanderbilt from 2014-2015 season

Trying to pull stats for Jarred Vanderbilt from 2013-2014 season

Trying to pull stats for Jarred Vanderbilt from 2012-2013 season

Trying to pull stats for Jarred Vanderbilt from 2011-2012 season

Trying to pull stats for Jarred 


Trying to pull stats for Justin Patton from 2016-2017 season

Trying to pull stats for Justin Patton from 2015-2016 season

Trying to pull stats for Justin Patton from 2014-2015 season

Trying to pull stats for Justin Patton from 2013-2014 season

Trying to pull stats for Justin Patton from 2012-2013 season

Trying to pull stats for Justin Patton from 2011-2012 season

Trying to pull stats for Justin Patton from 2010-2011 season

Trying to pull stats for Justin Patton from 2019-2020 season
No playoff games for 2019-2020 season
https://www.basketball-reference.com/players/t/trentga01.html
Incorrect ID tag for Gary Trent Jr., trying counter 02
https://www.basketball-reference.com/players/t/trentga02.html
Incorrect ID tag for Gary Trent Jr., trying counter 03
https://www.basketball-reference.com/players/t/trentga03.html
Page not found, trying google

Trying to pull stats for Gary Trent Jr. from 2018-2019 season

Trying to pull stats for Gary Trent Jr. from 2019-2020 season
No playoff gam

...


Trying to pull stats for Keita Bates-Diop from 2013-2014 season

Trying to pull stats for Keita Bates-Diop from 2012-2013 season

Trying to pull stats for Keita Bates-Diop from 2011-2012 season

Trying to pull stats for Keita Bates-Diop from 2010-2011 season
https://www.basketball-reference.com/players/r/reidna01.html
Correct ID tag for Naz Reid

Trying to pull stats for Naz Reid from 2019-2020 season
No playoff games for 2019-2020 season

Trying to pull stats for Naz Reid from 2018-2019 season

Trying to pull stats for Naz Reid from 2017-2018 season

Trying to pull stats for Naz Reid from 2016-2017 season

Trying to pull stats for Naz Reid from 2015-2016 season

Trying to pull stats for Naz Reid from 2014-2015 season

Trying to pull stats for Naz Reid from 2013-2014 season

Trying to pull stats for Naz Reid from 2012-2013 season

Trying to pull stats for Naz Reid from 2011-2012 season

Trying to pull stats for Naz Reid from 2010-2011 season
https://www.basketball-reference.com/player


Trying to pull stats for Cameron Reddish from 2017-2018 season

Trying to pull stats for Cameron Reddish from 2016-2017 season

Trying to pull stats for Cameron Reddish from 2015-2016 season

Trying to pull stats for Cameron Reddish from 2014-2015 season

Trying to pull stats for Cameron Reddish from 2013-2014 season

Trying to pull stats for Cameron Reddish from 2012-2013 season

Trying to pull stats for Cameron Reddish from 2011-2012 season

Trying to pull stats for Cameron Reddish from 2010-2011 season
https://www.basketball-reference.com/players/h/hernade01.html
Correct ID tag for Dewan Hernandez

Trying to pull stats for Dewan Hernandez from 2019-2020 season
No playoff games for 2019-2020 season

Trying to pull stats for Dewan Hernandez from 2018-2019 season

Trying to pull stats for Dewan Hernandez from 2017-2018 season

Trying to pull stats for Dewan Hernandez from 2016-2017 season

Trying to pull stats for Dewan Hernandez from 2015-2016 season

Trying to pull stats for Dewan H

No playoff games for 2019-2020 season

Trying to pull stats for Nassir Little from 2018-2019 season

Trying to pull stats for Nassir Little from 2017-2018 season

Trying to pull stats for Nassir Little from 2016-2017 season

Trying to pull stats for Nassir Little from 2015-2016 season

Trying to pull stats for Nassir Little from 2014-2015 season

Trying to pull stats for Nassir Little from 2013-2014 season

Trying to pull stats for Nassir Little from 2012-2013 season

Trying to pull stats for Nassir Little from 2011-2012 season

Trying to pull stats for Nassir Little from 2010-2011 season
https://www.basketball-reference.com/players/g/gaffoda01.html
Correct ID tag for Daniel Gafford

Trying to pull stats for Daniel Gafford from 2019-2020 season
No playoff games for 2019-2020 season

Trying to pull stats for Daniel Gafford from 2018-2019 season

Trying to pull stats for Daniel Gafford from 2017-2018 season

Trying to pull stats for Daniel Gafford from 2016-2017 season

Trying to pull st


Trying to pull stats for Bruno Fernando from 2012-2013 season

Trying to pull stats for Bruno Fernando from 2011-2012 season

Trying to pull stats for Bruno Fernando from 2010-2011 season
https://www.basketball-reference.com/players/m/mathega01.html
Correct ID tag for Garrison Mathews

Trying to pull stats for Garrison Mathews from 2019-2020 season
No playoff games for 2019-2020 season

Trying to pull stats for Garrison Mathews from 2018-2019 season

Trying to pull stats for Garrison Mathews from 2017-2018 season

Trying to pull stats for Garrison Mathews from 2016-2017 season

Trying to pull stats for Garrison Mathews from 2015-2016 season

Trying to pull stats for Garrison Mathews from 2014-2015 season

Trying to pull stats for Garrison Mathews from 2013-2014 season

Trying to pull stats for Garrison Mathews from 2012-2013 season

Trying to pull stats for Garrison Mathews from 2011-2012 season

Trying to pull stats for Garrison Mathews from 2010-2011 season
https://www.basketball-re


Trying to pull stats for Jalen Brunson from 2013-2014 season

Trying to pull stats for Jalen Brunson from 2012-2013 season

Trying to pull stats for Jalen Brunson from 2011-2012 season

Trying to pull stats for Jalen Brunson from 2010-2011 season
https://www.basketball-reference.com/players/b/bowmaky01.html
Correct ID tag for Ky Bowman

Trying to pull stats for Ky Bowman from 2019-2020 season
No playoff games for 2019-2020 season

Trying to pull stats for Ky Bowman from 2018-2019 season

Trying to pull stats for Ky Bowman from 2017-2018 season

Trying to pull stats for Ky Bowman from 2016-2017 season

Trying to pull stats for Ky Bowman from 2015-2016 season

Trying to pull stats for Ky Bowman from 2014-2015 season

Trying to pull stats for Ky Bowman from 2013-2014 season

Trying to pull stats for Ky Bowman from 2012-2013 season

Trying to pull stats for Ky Bowman from 2011-2012 season

Trying to pull stats for Ky Bowman from 2010-2011 season
https://www.basketball-reference.com/player

In [21]:
injs_df = pd.DataFrame(injs_df_arr,columns=["Name","Height [cm]","Weight [kg]","Position","Pos3","Shoots","Date","Age","Opp","MP","FG","FGA","FG%","3P","3PA",\
                                            "3P%","FT","FTA","FT%","ORB","DRB","TRB","AST","STL","BLK","TOV","PF","PTS","GmSc","+/-","Was_Injured?","Notes"])
no_injs_df = pd.DataFrame(noinjs_df_arr,columns=["Name","Height [cm]","Weight [kg]","Position","Pos3","Shoots","Date","Age","Opp","MP","FG","FGA","FG%","3P","3PA",\
                                            "3P%","FT","FTA","FT%","ORB","DRB","TRB","AST","STL","BLK","TOV","PF","PTS","GmSc","+/-","Was_Injured?","Notes"])

In [22]:
print(injs_df.shape)
print(no_injs_df.shape)

(8222, 32)
(250071, 32)


In [23]:
full_df = pd.concat((injs_df,no_injs_df))

In [24]:
full_df = full_df.loc[full_df['Age']!=0]

#### Make sure we don't have any duplicate rows before saving

In [26]:
full_df.drop_duplicates(subset=['Name','Date'],inplace=True)

In [27]:
print("Number of Duplicate Rows = {}".format(full_df[full_df.duplicated(subset=['Name','Date'])].shape[0]))

Number of Duplicate Rows = 0


In [28]:
full_df.shape

(258267, 32)

In [29]:
full_df.to_csv('nba_games_2010_20.csv')